In [ ]:
%load_ext autoreload
%autoreload 2
%run -n main.py

# sources

In [ ]:
# path = join_path(DATA_DIR, SOURCE)
# !mkdir -p {path}

In [ ]:
# for name in [NE5, BSNLP, FACTRU, GAREEV]:
#     path = join_path(CORUS_DATA_DIR, CORUS_FILES[name])
#     records = LOADS[name](path)
#     records = (_.adapted for _ in records)
#     records = log_progress(records, desc=name)

#     path = join_path(DATA_DIR, TARGET, name + JL + GZ)
#     items = (_.as_json for _ in records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
%run -n main.py
sources = {}
for name in [NE5, BSNLP, FACTRU, GAREEV]:
    path = join_path(DATA_DIR, SOURCE, name + JL + GZ)
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    sources[name] = [Markup.from_json(_) for _ in items]

# annotators

## cpu

In [ ]:
# for name in [MITIE, NATASHA, PULLENTI, TEXTERRA, TOMITA, DEEPPAVLOV, DEEPPAVLOV_BERT]:
#     path = join_path(DATA_DIR, name)
#     !mkdir -p {path}

In [ ]:
# %run -n main.py
# docker = docker_client()

In [ ]:
# %run -n main.py
# for ann_name in [MITIE, TOMITA, NATASHA, PULLENTI, TEXTERRA]:
#     log(ann_name)
#     ann = ANNS[ann_name]()
#     ann.start(docker)
#     ann.wait()

#     for source_name in [NE5, BSNLP, FACTRU, GAREEV]:
#         records = ann.map(_.text for _ in sources[source_name])
#         records = (_.adapted for _ in records)
#         records = log_progress(records, desc=source_name)

#         path = join_path(DATA_DIR, ann_name, source_name + JL + GZ)
#         items = (_.as_json for _ in records)
#         lines = format_jl(items)
#         dump_gz_lines(lines, path)
    
#     ann.stop(docker)

## gpu

In [ ]:
# for deeppavlov anns rent gpu
# select cuda 10.0, high bandwith
# !vast search offers | grep '1 x  GTX 1080 Ti'

In [ ]:
# %run -n main
# # ann = DeeppavlovAnnotator()
# ann = DeeppavlovBERTAnnotator()

In [ ]:
# !vast create instance 488497 --image {ann.image} --disk 20

In [ ]:
# !vast show instances

In [ ]:
# !ssh ssh4.vast.ai -p 12870 -l root -Nf -L {ann.port}:localhost:{ann.container_port}

In [ ]:
# for source_name in [NE5, BSNLP, FACTRU, GAREEV]:
#     records = sources[source_name]
#     records = log_progress(records, desc=source_name)
#     records = ann.map(_.text for _ in records)
#     records = (_.adapted for _ in records)

#     path = join_path(DATA_DIR, ann.name, source_name + JL + GZ)
#     items = (_.as_json for _ in records)
#     lines = format_jl(items)
#     dump_gz_lines(lines, path)

In [ ]:
# !vast destroy instance 492870

# score

In [ ]:
source_anns = {}
for source in SOURCES:
    for ann in ANNS:
        path = join_path(DATA_DIR, ann, source + JL + GZ)
        lines = load_gz_lines(path)
        items = parse_jl(lines)
        source_anns[source, ann] = [Markup.from_json(_) for _ in items]

In [ ]:
scores = {}
for source, ann in log_progress(source_anns):
    preds = source_anns[source, ann]
    targets = sources[source]
    scores[source, ann] = score_markups(preds, targets)

In [ ]:
# table = report_table(scores, SOURCES, ANNS)
# output = format_report(table)
# output

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="3" halign="left">factru</th>
      <th colspan="3" halign="left">gareev</th>
      <th colspan="3" halign="left">ne5</th>
      <th colspan="3" halign="left">bsnlp</th>
    </tr>
    <tr>
      <th>prec/recall/f1,%</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>deeppavlov</th>
      <td>86/96/91</td>
      <td>85/91/88</td>
      <td>72/75/74</td>
      <td>90/99/94</td>
      <td>-/-/-</td>
      <td>73/86/79</td>
      <td>94/93/94</td>
      <td>86/98/91</td>
      <td>87/88/88</td>
      <td>80/94/86</td>
      <td>63/95/76</td>
      <td>61/63/62</td>
    </tr>
    <tr>
      <th>deeppavlov_bert</th>
      <td>97/97/97</td>
      <td>91/94/92</td>
      <td>85/79/82</td>
      <td>96/99/98</td>
      <td>-/-/-</td>
      <td>88/94/91</td>
      <td>99/99/99</td>
      <td>98/99/99</td>
      <td>98/97/97</td>
      <td>93/97/95</td>
      <td>73/97/83</td>
      <td>79/69/74</td>
    </tr>
    <tr>
      <th>pullenti</th>
      <td>95/85/90</td>
      <td>86/77/81</td>
      <td>68/68/68</td>
      <td>91/96/93</td>
      <td>-/-/-</td>
      <td>63/64/63</td>
      <td>97/93/95</td>
      <td>86/85/86</td>
      <td>62/75/68</td>
      <td>91/88/90</td>
      <td>71/82/76</td>
      <td>58/55/56</td>
    </tr>
    <tr>
      <th>texterra</th>
      <td>91/88/90</td>
      <td>76/83/80</td>
      <td>86/45/59</td>
      <td>87/89/88</td>
      <td>-/-/-</td>
      <td>81/42/56</td>
      <td>97/83/90</td>
      <td>81/73/77</td>
      <td>83/46/59</td>
      <td>86/85/85</td>
      <td>67/92/78</td>
      <td>76/42/54</td>
    </tr>
    <tr>
      <th>tomita</th>
      <td>93/92/92</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>90/93/92</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>96/92/94</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>83/92/88</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
    </tr>
    <tr>
      <th>natasha</th>
      <td>96/78/86</td>
      <td>72/78/75</td>
      <td>41/23/29</td>
      <td>96/79/87</td>
      <td>-/-/-</td>
      <td>40/30/34</td>
      <td>98/75/85</td>
      <td>70/71/70</td>
      <td>49/32/39</td>
      <td>91/76/83</td>
      <td>65/88/75</td>
      <td>45/28/34</td>
    </tr>
    <tr>
      <th>mitie</th>
      <td>95/82/88</td>
      <td>90/81/86</td>
      <td>74/41/53</td>
      <td>85/84/84</td>
      <td>-/-/-</td>
      <td>55/38/45</td>
      <td>94/62/75</td>
      <td>74/56/64</td>
      <td>49/38/43</td>
      <td>81/66/73</td>
      <td>74/86/80</td>
      <td>55/49/52</td>
    </tr>
  </tbody>
</table>

In [ ]:
# output = format_github_report(table)
# html = output.to_html(escape=False)
# html = html.replace('border="1"', 'border="0"')
# print(html)